## Jupyter Lab Setup

To author documents in jupyter lab and generate HTML output, use the following YAML as a template.  This should be located in the first cell of the jupyter document, and the cell should be "raw".

```yaml
---
title: "metayer: authoring tools"
output: rmarkdown::html_vignette
vignette: >
  %\VignetteIndexEntry{metayer: authoring tools}
  %\VignetteEngine{knitr::rmarkdown}
  %\VignetteEncoding{UTF-8}
---
```

## metayer: vignette functionality

In [8]:
options(
  # cli.default_handler = default_handler,
  cli.default_handler = function(...) {},
  metayer.verbosity = 0
)

In [9]:
# load metayer
devtools::load_all()

ℹ Loading metayer


In [10]:
log_alert_info("hello world")

In [11]:
log_inform("hi hi")

hi hi


In [21]:
display_source("./tests/testthat")